In [1]:
import pickle 
import os
import pandas as pd
from tqdm import tqdm
import numpy as np

from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, balanced_accuracy_score

from d2c.benchmark import D2CWrapper

from d2c.descriptors.loader import DataLoader

from sklearn.ensemble import RandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier


In [2]:
with open('/home/jpalombarini/td2c/notebooks/contributions/model.pkl', 'rb') as f:
    model = pickle.load(f)

names = ['preprocessed_1.txt', 'preprocessed_2.txt'] 

ts_list = [np.loadtxt(os.path.join('/home/jpalombarini/td2c/notebooks/contributions/useful_data', name)) for name in names]

d2cwrapper = D2CWrapper(ts_list=ts_list, 
                        n_variables=[13,10], 
                        model=model, 
                        maxlags=1700, 
                        n_jobs=1, 
                        full=True, 
                        quantiles=True,
                        filename='d2c_results',
                        normalize=True, 
                        cmi='original', 
                        mb_estimator='original')



FileNotFoundError: [Errno 2] No such file or directory: '/home/jpalombarini/td2c/notebooks/contributions/model.pkl'

In [ ]:

d2cwrapper.run()

causal_df = d2cwrapper.get_causal_dfs()